# Music-Source-Separation-Training-Inference-Webui For Google Colab

For personal entertainment and non-commercial use only. Prohibited from use in content related to blood, gore, violence, sex, politics.<br>
Author: [Github@KitsuneX07](https://github.com/KitsuneX07) | [Github@SUC-DriverOld](https://github.com/SUC-DriverOld) | Gradio theme: [Gradio Theme](https://huggingface.co/spaces/NoCrypt/miku)<br>
Github: [MSST-WebUI](https://github.com/SUC-DriverOld/MSST-WebUI)

## Introduction

This is a webUI for [Music-Source-Separation-Training](https://github.com/ZFTurbo/Music-Source-Separation-Training), which is a music source separation training framework. You can use this webUI to infer the MSST model and UVR VR.Models (The inference code comes from [python-audio-separator](https://github.com/nomadkaraoke/python-audio-separator), and we do some changes on it), and the preset process page allows you to customize the processing flow yourself. When used on the cloud, you don't need to download the model yourself. WebUI will automatically download the model you need. Finally, we also provide some convenient tools such as [SOME: Vocals to MIDI](https://github.com/openvpi/SOME/) in the webUI.
[quick guide](https://r1kc63iz15l.feishu.cn/wiki/JSp3wk7zuinvIXkIqSUcCXY1nKc)
[model list](https://r1kc63iz15l.feishu.cn/wiki/Dy0bwG4XIizBgJkePDucILaMnlf)

In [1]:
#@title Connect to colab runtime and check GPU
#@markdown # Connect to colab runtime and check GPU
#@markdown

!nvidia-smi

Tue Oct 22 05:34:56 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             12W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
#@title Clone repository and install requirements
#@markdown # Clone repository and install requirements
#@markdown
#@markdown ### You may need to rerun this cell for a few times
#@markdown

!git clone https://github.com/SUC-DriverOld/MSST-WebUI.git

## config

In [ ]:
localproxy = "localtunnel" # ngrok localtunnel


In [ ]:
rom kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
civitai_token = user_secrets.get_secret("civitai_token")
hf_token = user_secrets.get_secret("hf_token")

if localproxy=="localtunnel":
    !npm install -g localtunnel
else:
    !pip install pyngrok
    ngrok_token = user_secrets.get_secret("ngrok_token")

In [9]:
%cd /kaggle/working/MSST-WebUI
%pip install tools/webUI_for_clouds/librosa-0.9.2-py3-none-any.whl
%pip install -r requirements.txt
%pip install -U fastapi==0.111.0


/kaggle/working/MSST-WebUI
Processing ./tools/webUI_for_clouds/librosa-0.9.2-py3-none-any.whl
librosa is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


In [6]:
#@title [Optional] Download SOME weights if needed
#@markdown # Download SOME weights if needed
#@markdown
#@markdown ### If you want to use "Vocals to MIDI", download the weights.
#@markdown
#@markdown ### "Vocals to MIDI" use project: [SOME](https://github.com/openvpi/SOME/).
#@markdown

!wget https://huggingface.co/Sucial/SOME_Models/resolve/main/model_steps_64000_simplified.ckpt -O /kaggle/working/MSST-WebUI/tools/SOME_weights/model_steps_64000_simplified.ckpt

--2024-10-22 05:38:23--  https://huggingface.co/Sucial/SOME_Models/resolve/main/model_steps_64000_simplified.ckpt
Resolving huggingface.co (huggingface.co)... 108.156.201.48, 108.156.201.102, 108.156.201.111, ...
Connecting to huggingface.co (huggingface.co)|108.156.201.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/29/18/2918b4c0d579576bfaf6c365a4df2509bb8f9745f1899b74d94e2b9e79f314d4/3a246c7063ec2856097004dcefa2093a8b6ccff7a6cde1cd49e8f3f227ede50e?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model_steps_64000_simplified.ckpt%3B+filename%3D%22model_steps_64000_simplified.ckpt%22%3B&Expires=1729834703&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyOTgzNDcwM319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzI5LzE4LzI5MThiNGMwZDU3OTU3NmJmYWY2YzM2NWE0ZGYyNTA5YmI4Zjk3NDVmMTg5OWI3NGQ5NGUyYjllNzlmMzE0ZDQvM2EyNDZjNzA2M2VjMjg1NjA5NzAwNGRjZWZhMjA5M2E4YjZ

In [5]:
%cd /kaggle/working/MSST-WebUI

import fileinput
print('found and replace')

with fileinput.FileInput("tools/webUI_for_clouds/webUI_for_clouds.py", inplace=True) as file:
    for line in file:
        if "app.launch(" in line:
            line = line.replace('app.launch(share=True)',
                                "app.launch(share=False)")
        print(line, end='')


/kaggle/working/MSST-WebUI
found and replace


In [8]:
#@title Initialize and start WebUI
#@markdown # Initialize and start WebUI
#@markdown
#@markdown ### Select the language you want to use for the WebUI.
#@markdown
#@markdown After running this cell, you can use your own device to connect to WebUI. The public link will be displayed below.
#@markdown
#@markdown If you want to modify the inference parameters of the preset process, edit the `data/webui_config.json` file.
#@markdown

%cd /kaggle/working/MSST-WebUI

LANGUAGE = "English" #@param ["Auto", "English", "简体中文", "繁體中文", "日本語", "😊"]
MODEL_DOWNLOAD_LINK = "huggingface.co" #@param ["Auto", "huggingface.co", "hf-mirror.com"]

import json
import os
import shutil

language_dict = {"Auto": "Auto", "简体中文": "zh_CN", "繁體中文": "zh_TW", "English": "en_US", "日本語": "ja_JP", "😊": "emoji"}

if not os.path.exists("data"):
    shutil.copytree("data_backup", "data")
if not os.path.exists("configs"):
    shutil.copytree("configs_backup", "configs")
os.makedirs("input", exist_ok=True)
os.makedirs("results", exist_ok=True)

with open("data/webui_config.json", 'r', encoding="utf-8") as f:
    config = json.load(f)
    config['settings']['language'] = language_dict[LANGUAGE]
    config['settings']['download_link'] = MODEL_DOWNLOAD_LINK
with open("data/webui_config.json", 'w', encoding="utf-8") as f:
    json.dump(config, f, indent=4)



In [ ]:
# start webui 
%cd /kaggle/working/MSST-WebUI

if localproxy=="localtunnel":
    import subprocess
    import urllib.request
    print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
    subprocess.Popen(["lt", "--port", "7860"])
    !python tools/webUI_for_clouds/webUI_for_clouds.py    
elif  localproxy=="ngrok":
    import os
    from pyngrok import ngrok
    from IPython.display import clear_output, Javascript
    ngrok.kill()
    ngrok.set_auth_token(ngrok_token)
    p_tunnel = ngrok.connect(7860)
    clear_output()
    print("webui Url:", p_tunnel.public_url)
    print("Save the link for later, this will take a while...")
    !python tools/webUI_for_clouds/webUI_for_clouds.py